# **Empowering Healthcare with Symbolic Learning and Knowledge Graph Embeddings**

![Desing Pattern](https://github.com/SDM-TIB/HyAI/assets/25593410/42cf771d-b82d-4097-b0f7-0ceb3581b171)

## **Pattern Detection, Analysis and Explanation over Lung Cancer Knowledge Graph**

>> **Pattern Detection. Unsupervised learning**
*   Learned embeddings of the entities in the Knowledge Graph
*   Apply community detection Algorithms: SemEP, KMeans
*   Compute the quality of the communities generated
*   Generate Radar-plots 


Git clone

In [ ]:
!git clone https://github.com/SDM-TIB/SymbolicLearning_KGE.git
%cd SymbolicLearning_KGE/PatternDetection

Install rdflib

In [ ]:
!pip install rdflib

In [ ]:
import ComputeCommunities as SemCD
import EvaluationMetric
import StatisticKG

**Clean the build**

In [1]:
!(cd src && make clean)

rm -rf ../semEP-node *.o *.dSYM ../*.dSYM *~


process_begin: CreateProcess(NULL, rm -rf ../semEP-node *.o *.dSYM ../*.dSYM *~, ...) failed.
make (e=2): The system cannot find the file specified.

make: *** [Makefile:31: clean] Error 2


In [1]:
!make clean

" **** Cleaning all in `pwd`  **** "
(cd src/ ; C:/ProgramData/chocolatey/lib/make/tools/install/bin/make.exe clean)


process_begin: CreateProcess(NULL, (cd src/ ; C:/ProgramData/chocolatey/lib/make/tools/install/bin/make.exe clean), ...) failed.
make (e=2): The system cannot find the file specified.

make: *** [Makefile:12: clean] Error 2


**Build the project**

In [ ]:
!make

**Statistics of Lung Cancer Knowledge Graph** 

In [ ]:
kg_name = 'TransformedKG'
StatisticKG.plot_statistics(kg_name)

## Run the semEP-node

**Parameter input**

In [ ]:
model_list = ['TransH', 'RotatE']
threshold = [35, 36, 37, 64, 65]
kg_name = 'TransformedKG'
target_predicate = 'hasRelapse_Progression'

In [ ]:
SemCD.run(kg_name=kg_name, target_predicate=target_predicate,
          model_list=model_list, threshold=threshold)

## Quality of the generated communities 

* Conductance (**InvC**): measures relatedness of entities in a community, and how different they are to entities outside the community.
* Performance (**P**): sums up the number of intra-community relationships, plus the number of non-existent relationships between communities.
* Total Cut (**InvTC**): sums up all similarities among entities in different communities.
* Modularity (**M**): measures the strength of the division of a graph into communities by comparing the density of connections inside communities to density of connections between different communities.
* Coverage (**Co**): compares the fraction of intra-community similarities between entities to the sum of all similarities between entities.

Moving to clusteringMeasures folder 

In [ ]:
%cd clusteringMeasures

**Clean the build**

In [ ]:
!make clean

**Build the project**

In [ ]:
!make

**Compute Metrices**

In [ ]:
import ComputeMetrices
ComputeMetrices.run(model_list, threshold)

**Generate Radar Plot**

In [ ]:
%cd ../
EvaluationMetric.GenerateRadarPlot(model_list, threshold)

# Analysis and Explanation over Lung Cancer Knowledge Graph

>> **Pattern Analysis and Explanation**
*   Visualize PCA Projection to 2D of the detected cummunities
*   Selecting a target predicate to analyze
*   Distribution of Relapse by Community


In [ ]:
%%capture
%cd ../PatternAnalysisExplanation
!pip install PyMuPDF
import PatternAnalysis
import pandas as pd

**Visualize PCA Projection to 2D of the detected cummunities**

In [ ]:
PatternAnalysis.PCA_projection(kg_name, model_list[0], threshold[1])
PatternAnalysis.PCA_projection(kg_name, model_list[1], threshold[3])

**Targeting the clusters with the '*target_predicate*' selected**

In [ ]:
target_cls = PatternAnalysis.target_cluster(kg_name, model_list[1], target_predicate, 'SemEP', threshold[3])
display(target_cls.head(), target_cls.shape)

**Compute the amount of patients for each the target value (column Relapse) and community**

In [ ]:
df = target_cls[['Relapse', 'Community']]
q = df.groupby(['Relapse', 'Community']).size().reset_index(name='count_values')
df_reset = q.reset_index(drop=True)
df_reset

**Normalized Clinical Records** 

In [ ]:
# Group by 'Relapse' and sum 'count_values'
result = df_reset.groupby('Relapse')['count_values'].sum().reset_index()

a = df_reset.loc[df_reset.Relapse=='No relapse'].copy()
b = df_reset.loc[df_reset.Relapse=='Relapse'].copy()
c = df_reset.loc[df_reset.Relapse=='UnKnown'].copy()
a['count_values'] = a['count_values']/result[result['Relapse'] == 'No relapse']['count_values'].values[0]
b['count_values'] = b['count_values']/result[result['Relapse'] == 'Relapse']['count_values'].values[0]
c['count_values'] = c['count_values']/result[result['Relapse'] == 'UnKnown']['count_values'].values[0]

df_reset = pd.concat([a, b, c])
df_reset

## **Visualization**

In [ ]:
PatternAnalysis.catplot(df_reset, model_list[1])